In [83]:
import numpy as np
from sklearn.svm import SVC
import torch
from sklearn.model_selection import KFold, StratifiedKFold
from itertools import product
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix

In [84]:
def mean_euclidean_error(vectors1, vectors2):
    """
    Compute the mean Euclidean error between two sets of 3D vectors.

    Parameters:
    - vectors1: NumPy array of shape (N, 3) representing the first set of 3D vectors
    - vectors2: NumPy array of shape (N, 3) representing the second set of 3D vectors

    Returns:
    - mean_error: Mean Euclidean error between the two sets of vectors
    """
    # Check if the input arrays have the correct shape
    if vectors1.shape != vectors2.shape or vectors1.shape[1] != 3:
        raise ValueError("Input arrays must be of shape (N, 3)")

    # Compute Euclidean distance
    euclidean_distance = np.linalg.norm(vectors1 - vectors2, axis=1)

    # Calculate the mean Euclidean error
    mean_error = np.mean(euclidean_distance)

    return mean_error

In [85]:
def preprocess_monk(file_name):
    '''
    Load data from the Monk dataset and preprocess using one-hot encoding.

    Parameters:
    - file_name (str): The file name of the dataset.

    Returns:
    - x (torch.Tensor): Input data after one-hot encoding.
    - y (torch.Tensor): Target data.
    '''

    # load the dataset, split into input (X) and output (y) variables
    df = pd.read_csv(file_name, delimiter=' ', header=None,
                     names=['_','target', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'data_number'],
                     index_col=False)

    # Extract input features and target variable
    x1, x2, x3, x4, x5, x6, target = (np.array(df[feature]) for feature in ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'target'])

    # Initialize OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False)

    # Reshape and transform x1 using one-hot encoding
    input_one_hot = encoder.fit_transform(x1.reshape(-1, 1))

    # Loop through the remaining input features and concatenate one-hot encoded values
    for x in [x2,x3,x4,x5,x6]:
        data =x.reshape(-1, 1)
        one_hot_encoded = encoder.fit_transform(data)
        input_one_hot = np.hstack((input_one_hot, one_hot_encoded))

    # Convert to PyTorch tensors
    x = torch.tensor(input_one_hot, dtype=torch.float32)#.cuda()
    y = torch.tensor(target, dtype=torch.float32).reshape(-1,1)#.cuda()

    return x, y

In [86]:
from sklearn.model_selection import train_test_split
# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('ML-CUP23-TR.csv', delimiter=',')
X = dataset[:,1:11]
y = dataset[:,11:14]



# Split the data into training and testing sets (80%/20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [87]:
def perform_grid_search_kfold(kernels, Cs, k_folds, x, y):
    '''
    Perform grid search with k-fold cross-validation for Monk classifier hyperparameters.

    Parameters:
    - hidden_sizes (list): List of hidden layer sizes to explore.
    - learning_rates (list): List of learning rates to explore.
    - momentums (list): List of momentum values to explore.
    - batch_sizes (list): List of batch sizes to explore.
    - reg_coeffs (list): List of regularization coefficients to explore.
    - k_folds (int): Number of folds for cross-validation.
    - x (torch.Tensor): Input data.
    - y (torch.Tensor): Target data.
    - plot_curves (bool): Whether to plot training curves for each hyperparameter combination (default: False).

    Returns:
    - best_hyperparams (list): List of best hyperparameters based on highest average validation accuracy.
    '''

    best_acc = 0
    best_hyperparams = []

    for kernel, C in product(kernels, Cs):
        print('kernel:', kernel, 'C:',C)

        kf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

        # Lists to store training and validation losses and accuracies for each epoch
        val_acc = []
        svm_classifier_rbf = SVC(kernel=kernel, C=C)
        # Perform K-fold cross-validation
        for fold, (train_indices, val_indices) in enumerate(kf.split(x,y)):
            #print(f"\nFold {fold + 1}/{k_folds}")

            # Split the data into training and validation (or test) sets
            X_train, X_val = x[train_indices], x[val_indices]
            Y_train, Y_val = y[train_indices], y[val_indices]

            svm_classifier_rbf.fit(X_train, Y_train)
            y_pred_rbf = svm_classifier_rbf.predict(X_val)
            accuracy_rbf = accuracy_score(Y_val, y_pred_rbf)

            val_acc.append(accuracy_rbf)

        print(f'{kernel}, C={C}, acc={np.mean(val_acc)}+-{np.std(val_acc)}')


        if np.mean(val_acc) >= best_acc:
            best_acc = np.mean(val_acc)
            best_hyperparams = [kernel,C]

    print(best_hyperparams)
    return best_hyperparams

# MONK 1

In [88]:
x_train, y_train = preprocess_monk(file_name='monk_data/monks-1.train')
x_test, y_test = preprocess_monk(file_name='monk_data/monks-1.test')

In [89]:
kernels = ['rbf']
Cs = [0.01,0.1,1,100,1000,10000,100000]
print(Cs)
best_hyperparams = perform_grid_search_kfold(kernels,
                          Cs,
                          k_folds=3,
                          x=x_train,
                          y=y_train.ravel())

[0.01, 0.1, 1, 100, 1000, 10000, 100000]
kernel: rbf C: 0.01
rbf, C=0.01, acc=0.5553619821912504+-0.09554017291293267
kernel: rbf C: 0.1
rbf, C=0.1, acc=0.5553619821912504+-0.09554017291293267
kernel: rbf C: 1
rbf, C=1, acc=0.8070073557878435+-0.04973517181408441
kernel: rbf C: 100
rbf, C=100, acc=0.8877274487030585+-0.06814882183447475
kernel: rbf C: 1000
rbf, C=1000, acc=0.8877274487030585+-0.06814882183447475
kernel: rbf C: 10000
rbf, C=10000, acc=0.8877274487030585+-0.06814882183447475
kernel: rbf C: 100000
rbf, C=100000, acc=0.8877274487030585+-0.06814882183447475
['rbf', 100000]


In [90]:
svm_classifier_rbf = SVC(kernel = best_hyperparams[0], C = best_hyperparams[1])
svm_classifier_rbf.fit(x_train,y_train.ravel())
# Make predictions on the test set
y_pred_rbf = svm_classifier_rbf.predict(x_test)
# Calculate accuracy
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)
print("Accuracy with RBF kernel:", accuracy_rbf)

Accuracy with RBF kernel: 0.9953703703703703


# MONK 2

In [95]:
x_train, y_train = preprocess_monk(file_name='monk_data/monks-2.train')
x_test, y_test = preprocess_monk(file_name='monk_data/monks-2.test')

In [96]:
kernels = ['sigmoid', 'poly', 'rbf', 'linear']
Cs = [0.01,0.1,1,100,1000,10000,100000]
print(Cs)
best_hyperparams = perform_grid_search_kfold(kernels,
                          Cs,
                          k_folds=3,
                          x=x_train,
                          y=y_train.ravel())

[0.01, 0.1, 1, 100, 1000, 10000, 100000]
kernel: sigmoid C: 0.01
sigmoid, C=0.01, acc=0.6213450292397661+-0.005168909219199899
kernel: sigmoid C: 0.1
sigmoid, C=0.1, acc=0.6213450292397661+-0.005168909219199899
kernel: sigmoid C: 1
sigmoid, C=1, acc=0.4737886382623225+-0.0542949917799745
kernel: sigmoid C: 100
sigmoid, C=100, acc=0.3670634920634921+-0.02494008946423916
kernel: sigmoid C: 1000
sigmoid, C=1000, acc=0.39066416040100255+-0.017335633248442263
kernel: sigmoid C: 10000
sigmoid, C=10000, acc=0.37896825396825395+-0.033082007937565605
kernel: sigmoid C: 100000
sigmoid, C=100000, acc=0.39066416040100255+-0.017335633248442263
kernel: poly C: 0.01
poly, C=0.01, acc=0.6213450292397661+-0.005168909219199899
kernel: poly C: 0.1
poly, C=0.1, acc=0.6213450292397661+-0.005168909219199899
kernel: poly C: 1
poly, C=1, acc=0.6747076023391813+-0.05414293621971112
kernel: poly C: 100
poly, C=100, acc=0.698203842940685+-0.043812880343637436
kernel: poly C: 1000
poly, C=1000, acc=0.698203842940

In [98]:
svm_classifier_rbf = SVC(kernel = best_hyperparams[0], C = best_hyperparams[1])
svm_classifier_rbf.fit(x_train,y_train.ravel())
# Make predictions on the test set
y_pred_rbf = svm_classifier_rbf.predict(x_test)
print(y_pred_rbf)
# Calculate accuracy
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)
print("Accuracy with RBF kernel:", accuracy_rbf)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1.
 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1.
 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.